# Finite Difference Operators and C Kernels in NRPy

## Author: Zach Etienne

## This notebook explores how NRPy's `finite_difference` infrastructure converts analytic derivative operators into finite-difference (FD) coefficients, memory access patterns, and reusable C functions. We start from the Taylor-series construction of centered finite differences, then connect this directly to the core helper functions implemented in `finite_difference.py`. Along the way we examine centered, upwinded, and Kreiss Oliger (KO) operators, how gridfunctions are read from memory (with optional SIMD support), and how prototype FD operators are wrapped into standalone C functions.

### Required reading if you are unfamiliar with programming or [computer algebra systems](https://en.wikipedia.org/wiki/Computer_algebra_system). Otherwise, use for reference; you should be able to pick up the syntax as you follow the tutorial.
+ **[Python Tutorial](https://docs.python.org/3/tutorial/index.html)**
+ **[SymPy Tutorial](http://docs.sympy.org/latest/tutorial/intro.html)**

### NRPy Source Code for this module:  
* [finite_difference.py](../edit/finite_difference.py)

# Table of Contents

The module is organized as follows:

1. [Step 1](#Step-1:-Initialize-core-Python/NRPy-modules-for-finite-differencing): Initialize core Python/NRPy modules for finite differencing
1. [Step 2](#Step-2:-From-Taylor-series-to-centered-finite-difference-coefficients): From Taylor series to centered finite difference coefficients
1. [Step 3](#Step-3:-Using-setup_FD_matrix__return_inverse-to-build-FD-matrices): Using `setup_FD_matrix__return_inverse` to build FD matrices
1. [Step 4](#Step-4:-compute_fdcoeffs_fdstencl:-coefficients-and-stencil-layouts-for-many-derivatives): `compute_fdcoeffs_fdstencl`: coefficients and stencil layouts for many derivatives
1. [Step 5](#Step-5:-From-SymPy-derivatives-to-gridfunctions-and-derivative-operators): From SymPy derivatives to gridfunctions and derivative operators
1. [Step 6](#Step-6:-Reading-gridfunctions-from-memory-with-read_gfs_from_memory): Reading gridfunctions from memory with `read_gfs_from_memory`
1. [Step 7](#Step-7:-Building-prototype-FD-operators-with-proto_FD_operators_to_sympy_expressions): Building prototype FD operators with `proto_FD_operators_to_sympy_expressions`
1. [Step 8](#Step-8:-Wrapping-FD-expressions-into-reusable-C-functions): Wrapping FD expressions into reusable C functions

# Step 1: Initialize core Python/NRPy modules for finite differencing
### \[Back to [top](#Table-of-Contents)\]

We begin by importing the core Python and NRPy modules used throughout this notebook:

* `sympy` for symbolic algebra.
* `nrpy.params` for the global parameter interface.
* `nrpy.grid` for gridfunction registration and metadata.
* `nrpy.indexedexp` for declaring indexed SymPy arrays (for example tensors and derivative objects).
* `nrpy.c_function` for generating standalone C functions.
* `nrpy.finite_difference` for all of the FD helper routines discussed in this notebook.

We also set a default infrastructure so that SIMD aware code paths behave as in production NRPy setups.

In [1]:
# Step 1: Initialize core Python/NRPy modules for finite differencing
import sympy as sp

import nrpy.params as par          # NRPy: parameter interface
import nrpy.grid as gri            # NRPy: numerical grid and gridfunctions
import nrpy.indexedexp as ixp      # NRPy: indexed SymPy expressions
import nrpy.c_function as cfc      # NRPy: thin wrapper around C functions
import nrpy.finite_difference as fd

# Set the infrastructure in a way that is compatible with SIMD and FD helpers
par.set_parval_from_str("Infrastructure", "BHaH")

# Optionally override the default FD order parameter (finite_difference.py registers this too)
par.set_parval_from_str("fd_order", 4)

# Step 2: From Taylor series to centered finite difference coefficients
### \[Back to [top](#Table-of-Contents)\]

Before diving into the helper functions in `finite_difference.py`, it is helpful to recall how a standard centered finite difference stencil arises from the Taylor expansion of a smooth function. This section follows the analytic derivation that motivates the matrix inversion strategy used by NRPy.

## Example: Centered Finite Difference Representation of $u'(x_0) = u'_0$ Accurate to Fourth Order in $\Delta x$

As an illustration, consider a uniform grid and derive the centered finite difference coefficients for the first derivative that are accurate to fourth order in $\Delta x$. The fourth order accurate, uniformly sampled, centered finite difference approximation to $u'(x_0)$ is equivalent to taking the derivative of the unique polynomial that interpolates $u(x)$ at the sampled points
$\{ x_{-2}, x_{-1}, x_{0}, x_{1}, x_{2} \}$, where $x_i = x_0 + i \Delta x$.

The Taylor series expansion of a function $u(x)$ about a point $x_0$ is

$$u(x) = \sum_{n=0}^\infty \frac{u^{(n)}(x_0)}{n!} (x-x_0)^n,$$

where $u^{(n)}(x_0)$ is the $n$th derivative of $u$ evaluated at $x_0$. Based on this, we can write the Taylor expansion of $u$ at a point $x = x_0 + j \Delta x$ as

\begin{align}
u(x_0 + j \Delta x) &= \sum_{n=0}^\infty \frac{u^{(n)}(x_0)}{n!} (j \Delta x)^n, \text{ or equivalently:} \\
u_j &= \sum_{n=0}^\infty \frac{u^{(n)}_0}{n!} (j \Delta x)^n.
\end{align}

Our goal is to compute $u^{(1)}(x_0) = u'_0$ at some point $x_0$, with a leading truncation error proportional to $(\Delta x)^4$. We accomplish this by Taylor expanding $u(x_j)$ about $x = x_0$ for $j \in \{-2,-1,0,1,2\}$, each up to the $n = 5$ term:

\begin{align}
u_{-2} &= u_0 - (2 \Delta x) u'_0 + \frac{(2 \Delta x)^2}{2} u''_0 - \frac{(2 \Delta x)^3}{3!} u'''_0 + \frac{(2 \Delta x)^4}{4!} u^{(4)}_0 + \mathcal{O}\big((\Delta x)^5\big) \\
u_{-1} &= u_0 - (\Delta x) u'_0 + \frac{(\Delta x)^2}{2} u''_0 - \frac{(\Delta x)^3}{3!} u'''_0 + \frac{(\Delta x)^4}{4!} u^{(4)}_0 + \mathcal{O}\big((\Delta x)^5\big)\\
u_{0} &= u_0 \\
u_{1} &= u_0 + (\Delta x) u'_0 + \frac{(\Delta x)^2}{2} u''_0 + \frac{(\Delta x)^3}{3!} u'''_0 + \frac{(\Delta x)^4}{4!} u^{(4)}_0 + \mathcal{O}\big((\Delta x)^5\big)\\
u_{2} &= u_0 + (2 \Delta x) u'_0 + \frac{(2 \Delta x)^2}{2} u''_0 + \frac{(2 \Delta x)^3}{3!} u'''_0 + \frac{(2 \Delta x)^4}{4!} u^{(4)}_0 + \mathcal{O}\big((\Delta x)^5\big).\\
\end{align}

We now combine the above equations to find coefficients $a_j$ such that
$$
\frac{a_{-2} u_{-2} + a_{-1} u_{-1} + a_0 u_0 + a_{1} u_{1} + a_{2} u_{2}}{\Delta x}
= u'_0 + \mathcal{O}\big((\Delta x)^4\big).
$$

Substituting the Taylor expansions into the left hand side gives

\begin{align}
& \frac{a_{-2} u_{-2} + a_{-1} u_{-1} + a_0 u_0 + a_{1} u_{1} + a_{2} u_{2}}{\Delta x} \\
= & \left( u_0 - (2 \Delta x) u'_0 + \frac{(2 \Delta x)^2}{2} u''_0 - \frac{(2 \Delta x)^3}{3!} u'''_0 + \frac{(2 \Delta x)^4}{4!} u^{(4)}_0 \right) a_{-2} \\
& + \left( u_0 - (\Delta x) u'_0 + \frac{(\Delta x)^2}{2} u''_0 - \frac{(\Delta x)^3}{3!} u'''_0 + \frac{(\Delta x)^4}{4!} u^{(4)}_0 \right) a_{-1} \\
& + \left( u_0 \right) a_{0} \\
& + \left( u_0 + (\Delta x) u'_0 + \frac{(\Delta x)^2}{2} u''_0 + \frac{(\Delta x)^3}{3!} u'''_0 + \frac{(\Delta x)^4}{4!} u^{(4)}_0 \right) a_{1} \\
& + \left( u_0 + (2 \Delta x) u'_0 + \frac{(2 \Delta x)^2}{2} u''_0 + \frac{(2 \Delta x)^3}{3!} u'''_0 + \frac{(2 \Delta x)^4}{4!} u^{(4)}_0 \right) a_{2}.
\end{align}

Each derivative in the Taylor expansion comes with a factor of $\Delta x$, so it is convenient to temporarily absorb these powers of $\Delta x$ into the derivatives (we will restore them later) and group by derivative order:

\begin{align}
& a_{-2} u_{-2} + a_{-1} u_{-1} + a_0 u_0 + a_{1} u_{1} + a_{2} u_{2} \\
& = \left( a_{-2} + a_{-1} + a_0 + a_{1} + a_{2} \right) u_0 \\
& + \left( -2 a_{-2} - a_{-1} + a_{1} + 2 a_{2} \right) u'_0 \\
& + \left( 2^2 a_{-2} + a_{-1} + a_{1} + 2^2 a_{2} \right) \frac{u''_0}{2!} \\
& + \left( -2^3 a_{-2} - a_{-1} + a_{1} + 2^3 a_{2} \right) \frac{u'''_0}{3!} \\
& + \left( 2^4 a_{-2} + a_{-1} + a_{1} + 2^4 a_{2} \right) \frac{u^{(4)}_0}{4!}.
\end{align}

For this to match $u'_0$ for all choices of $\{u_0,u'_0,u''_0,u'''_0,u^{(4)}_0\}$, we require

\begin{align}
0 &= a_{-2} + a_{-1} + a_0 + a_{1} + a_{2},\\
1 &= -2 a_{-2} - a_{-1} + a_{1} + 2 a_{2},\\
0 \times 2! &= 2^2 a_{-2} + a_{-1} + a_{1} + 2^2 a_{2},\\
0 \times 3! &= -2^3 a_{-2} - a_{-1} + a_{1} + 2^3 a_{2}, \\
0 \times 4! &= 2^4 a_{-2} + a_{-1} + a_{1} + 2^4 a_{2}.
\end{align}

Writing this as a matrix equation (with $0! = 1$) yields

\begin{equation}
\left(
\begin{array}{c}
0 \times 0! \\
1 \times 1! \\
0 \times 2! \\
0 \times 3! \\
0 \times 4! \\
\end{array}
\right)
=
\left(
\begin{array}{ccccc}
 1 &  1 & 1 & 1 & 1 \\
(-2)^1 &(-1)^1 & 0 & 1 & 2 \\
(-2)^2 &(-1)^2 & 0 & 1 & 2^2 \\
(-2)^3 &(-1)^3 & 0 & 1 & 2^3 \\
(-2)^4 &(-1)^4 & 0 & 1 & 2^4 \\
\end{array}
\right)
\left(
\begin{array}{c}
a_{-2} \\
a_{-1} \\
a_{0} \\
a_{1} \\
a_{2} \\
\end{array}
\right).
\end{equation}

Thus the computation of finite difference coefficients reduces to solving an $N \times N$ linear system. The structure of the matrix depends only on the stencil layout and not on $\Delta x$ itself.

The inverse of the above matrix is

\begin{equation}
\left(
\begin{array}{ccccc}
0 & 1/12 & -1/24 & -1/12 & 1/24 \\
0 & -2/3 & 2/3 & 1/6 & -1/6 \\
1 & 0 & -5/4 & 0 & 1/4 \\
0 & 2/3 & 2/3 & -1/6 & -1/6 \\
0 & -1/12 & -1/24 & 1/12 & 1/24 \\
\end{array}
\right)
\label{fourthorder_inv_matrix}.
\end{equation}

The coefficients for the $M$th derivative can be read off by multiplying the $(M+1)$st column by $M! / (\Delta x)^M$. For example, the zeroth derivative at $x_0$ is

$$\frac{0!}{(\Delta x)^0} \times (0 u_{-2} + 0 u_{-1} + u_0 + 0 u_{1} + 0 u_{2}) = u_0,$$

which is exact. The first derivative finite difference approximation at $x_0$ is

$$\frac{1!}{\Delta x} \times \left(\frac{1}{12}( u_{-2} - u_{2}) + \frac{2}{3}( -u_{-1} + u_{1})\right) \approx (\partial_x u)_0,$$

and the second derivative finite difference approximation at $x_0$ is

$$\frac{2!}{(\Delta x)^2} \times \left(-\frac{1}{24}(u_{-2} + u_{2}) + \frac{2}{3}(u_{-1} + u_{1}) - \frac{5}{4} u_0 \right) \approx (\partial_x^2 u)_0.$$

In short, this matrix yields the finite difference derivative coefficients with the smallest possible truncation error for a 5 point stencil. One can show by examining higher order terms in the Taylor expansions that the first and second derivative coefficients are correct to order $(\Delta x)^4$, while the third and fourth derivative coefficients are correct to order $(\Delta x)^2$.

### Exercise 1: Find the exact expressions for the leading (dominant) error term for all derivatives that can be computed from this matrix (zeroth through fourth derivatives).

### Exercise 2: Construct the matrix whose inverse yields the 5 point stencil upwinded derivative coefficients (i.e., the stencil includes points $\{u_{-4},u_{-3},u_{-2},u_{-1},u_{0}\}$).

NRPy implements exactly this simple matrix inversion strategy in `finite_difference.py` to evaluate finite difference coefficients for a wide variety of stencils.

# Step 3: Using setup_FD_matrix__return_inverse to build FD matrices
### \[Back to [top](#Table-of-Contents)\]

The function `setup_FD_matrix__return_inverse(stencil_width, UPDOWNWIND_stencil_shift)` in `finite_difference.py` constructs and inverts the matrix described above, for arbitrary stencil width and optional upwinding.

* `stencil_width` sets the number of points in the stencil.
* `UPDOWNWIND_stencil_shift` controls where the point $x_0$ sits inside the stencil:
  * `0` for centered stencils.
  * Positive values for upwinded stencils.
  * Negative values for downwinded stencils.

The function uses LU decomposition to invert the matrix efficiently and caches the inverse in a dictionary so subsequent calls with the same arguments are fast.

In [2]:
# Step 3: Demonstrate setup_FD_matrix__return_inverse for a 5-point centered stencil

# 5-point stencil (4th order accurate), centered on i = 0
stencil_width = 5
UPDOWNWIND_stencil_shift = 0

Minv_centered = fd.setup_FD_matrix__return_inverse(stencil_width, UPDOWNWIND_stencil_shift)
print("Inverse FD matrix for 5-point centered stencil (rows: derivative order, columns: stencil point):")
sp.pprint(Minv_centered)

# Extract the first- and second-derivative coefficients for comparison with the analytic result:
first_deriv_col = 1  # column index for first derivative in the inverted matrix
second_deriv_col = 2  # column index for second derivative

a_first = [sp.factorial(1) * Minv_centered[i, first_deriv_col] for i in range(stencil_width)]
a_second = [sp.factorial(2) * Minv_centered[i, second_deriv_col] for i in range(stencil_width)]

print("\nFirst derivative coefficients (scaled by 1!/dx):")
print(a_first)
print("\nSecond derivative coefficients (scaled by 2!/dx^2):")
print(a_second)

Inverse FD matrix for 5-point centered stencil (rows: derivative order, columns: stencil point):
⎡0  1/12   -1/24  -1/12  1/24⎤
⎢                            ⎥
⎢0  -2/3    2/3    1/6   -1/6⎥
⎢                            ⎥
⎢1    0    -5/4     0    1/4 ⎥
⎢                            ⎥
⎢0   2/3    2/3   -1/6   -1/6⎥
⎢                            ⎥
⎣0  -1/12  -1/24  1/12   1/24⎦

First derivative coefficients (scaled by 1!/dx):
[1/12, -2/3, 0, 2/3, -1/12]

Second derivative coefficients (scaled by 2!/dx^2):
[-1/12, 4/3, -5/2, 4/3, -1/12]


In [3]:
# Step 3 (continued): Demonstrate an upwinded stencil

# 5-point stencil, fully upwinded to the right (center shifts toward the left)
UPDOWNWIND_stencil_shift_up = int((stencil_width - 1) / 2)  # full upwind
Minv_upwind = fd.setup_FD_matrix__return_inverse(stencil_width, UPDOWNWIND_stencil_shift_up)

print("\nInverse FD matrix for 5-point fully upwinded stencil (right-going):")
sp.pprint(Minv_upwind)


Inverse FD matrix for 5-point fully upwinded stencil (right-going):
⎡   -25    35               ⎤
⎢1  ────   ──    -5/12  1/24⎥
⎢    12    24               ⎥
⎢                           ⎥
⎢0   4    -13/3   3/2   -1/6⎥
⎢                           ⎥
⎢0   -3   19/4    -2    1/4 ⎥
⎢                           ⎥
⎢0  4/3   -7/3    7/6   -1/6⎥
⎢                           ⎥
⎢          11               ⎥
⎢0  -1/4   ──    -1/4   1/24⎥
⎣          24               ⎦


# Step 4: compute_fdcoeffs_fdstencl: coefficients and stencil layouts for many derivatives
### \[Back to [top](#Table-of-Contents)\]

While `setup_FD_matrix__return_inverse` returns the full inverse matrix, most applications only need the coefficients and stencil offsets for specific derivatives. The helper

```python
fd.compute_fdcoeffs_fdstencl(derivstring, fd_order)
```

takes a string describing the derivative and the FD order, and returns:

* `fdcoeffs`: a list of SymPy `Rational` coefficients.
* `fdstencl`: a list of integer triples recording the offsets $(\Delta i_0, \Delta i_1, \Delta i_2)$ for each coefficient.

The `derivstring` encodes both derivative order and upwinding variant, for example:

* `"dD0"`: first derivative in direction 0, centered.
* `"dupD1"` / `"ddnD1"`: first derivative in direction 1, one point upwinded/downwinded.
* `"dfullupD2"` / `"dfulldnD2"`: first derivative in direction 2, fully upwinded/downwinded (all stencil points on one side).
* `"dDD01"`: second derivative with indices $0$ and $1$ (unmixed).
* `"dKOD0"`: Kreiss Oliger dissipation operator in direction 0.

The function also knows how to assemble mixed second derivatives from products of first-derivative stencils.

In [4]:
# Step 4: Sample calls to compute_fdcoeffs_fdstencl

# Use a 4th order derivative (stencil_width = fd_order + 1 = 5)
fd_order = 4

# First derivative, centered in direction 0
fdcoeffs_dD0, fdstencl_dD0 = fd.compute_fdcoeffs_fdstencl("dD0", fd_order)
print("Centered first derivative dD0 (4th order):")
print("  Coeffs:", fdcoeffs_dD0)
print("  Stencil offsets:", fdstencl_dD0)

# First derivative, single-point upwinded in direction 0
fdcoeffs_dupD0, fdstencl_dupD0 = fd.compute_fdcoeffs_fdstencl("dupD0", fd_order)
print("\nUpwinded first derivative dupD0 (4th order):")
print("  Coeffs:", fdcoeffs_dupD0)
print("  Stencil offsets:", fdstencl_dupD0)

# Second derivative, unmixed in direction 1
fdcoeffs_dD11, fdstencl_dD11 = fd.compute_fdcoeffs_fdstencl("dDD11", fd_order)
print("\nUnmixed second derivative dDD11 (4th order):")
print("  Coeffs:", fdcoeffs_dD11)
print("  Stencil offsets:", fdstencl_dD11)

# Mixed second derivative in directions 0 and 1
fdcoeffs_dD01, fdstencl_dD01 = fd.compute_fdcoeffs_fdstencl("dDD01", fd_order)
print("\nMixed second derivative dDD01 (4th order):")
print("  Coeffs:", fdcoeffs_dD01)
print("  Stencil offsets:", fdstencl_dD01)

# Kreiss Oliger derivative in direction 2 (uses highest-order row of Minv)
fdcoeffs_dKOD2, fdstencl_dKOD2 = fd.compute_fdcoeffs_fdstencl("dKOD2", fd_order)
print("\nKreiss Oliger dissipation operator dKOD2:")
print("  Coeffs:", fdcoeffs_dKOD2)
print("  Stencil offsets:", fdstencl_dKOD2)

Centered first derivative dD0 (4th order):
  Coeffs: [1/12, -2/3, 2/3, -1/12]
  Stencil offsets: [[-2, 0, 0], [-1, 0, 0], [1, 0, 0], [2, 0, 0]]

Upwinded first derivative dupD0 (4th order):
  Coeffs: [-1/4, -5/6, 3/2, -1/2, 1/12]
  Stencil offsets: [[-1, 0, 0], [0, 0, 0], [1, 0, 0], [2, 0, 0], [3, 0, 0]]

Unmixed second derivative dDD11 (4th order):
  Coeffs: [-1/12, 4/3, -5/2, 4/3, -1/12]
  Stencil offsets: [[0, -2, 0], [0, -1, 0], [0, 0, 0], [0, 1, 0], [0, 2, 0]]

Mixed second derivative dDD01 (4th order):
  Coeffs: [1/144, -1/18, 1/18, -1/144, -1/18, 4/9, -4/9, 1/18, 1/18, -4/9, 4/9, -1/18, -1/144, 1/18, -1/18, 1/144]
  Stencil offsets: [[-2, -2, 0], [-1, -2, 0], [1, -2, 0], [2, -2, 0], [-2, -1, 0], [-1, -1, 0], [1, -1, 0], [2, -1, 0], [-2, 1, 0], [-1, 1, 0], [1, 1, 0], [2, 1, 0], [-2, 2, 0], [-1, 2, 0], [1, 2, 0], [2, 2, 0]]

Kreiss Oliger dissipation operator dKOD2:
  Coeffs: [1/64, -3/32, 15/64, -5/16, 15/64, -3/32, 1/64]
  Stencil offsets: [[0, 0, -3], [0, 0, -2], [0, 0, -1], [0

# Step 5: From SymPy derivatives to gridfunctions and derivative operators
### \[Back to [top](#Table-of-Contents)\]

In real NRPy applications, you rarely construct derivative strings by hand. Instead, you declare derivative objects as SymPy arrays (for example `vU_dD` or `hDD_dupD`) and then let helper routines inspect the expressions and determine which derivative operators and base gridfunctions appear.

Two key helpers in `finite_difference.py` implement this logic:

* `extract_list_of_deriv_var_strings_from_sympyexpr_list(list_of_free_symbols, upwind_control_vec)`  
  Scans a list of SymPy symbols, filters out gridfunctions and code parameters, and returns only derivative symbols (matching patterns like `_dD`, `_dKOD`, `_dupD`, `_ddnD`). If `upwind_control_vec` is not a string, then for every `_dupD` symbol it also creates the corresponding `_ddnD` symbol, so that both sides of the upwind derivative are available.

* `extract_base_gfs_and_deriv_ops_lists__from_list_of_deriv_vars(list_of_deriv_vars)`  
  Splits each derivative symbol into:
  * A base gridfunction name (including tensor indices, such as `hDD01`).
  * A derivative operator string (such as `dD0`, `dupD2`, or `dDD01`).

The combination of these functions tells NRPy which FD stencils are needed and which gridfunctions they act on.

In [5]:
# Step 5: Demonstrate extraction of derivatives and base gridfunctions

# Reset the global gridfunction dictionary for a clean slate
gri.glb_gridfcs_dict.clear()

# Register a rank-1 vector gridfunction vU
vU = gri.register_gridfunctions_for_single_rank1("vU")

# Declare a rank-2 array of first derivatives vU_dD
vU_dD = ixp.declarerank2("vU_dD")

# Build a simple expression that uses both gridfunctions and their derivatives
expr = vU_dD[0][1] * vU[2] + vU_dD[2][0] * vU[1]

# Extract all free symbols
list_of_free_symbols = list(expr.free_symbols)

# Here we set upwind_control_vec to a string, so no automatic _ddnD creation
upwind_control_vec = "unset"

deriv_vars = fd.extract_list_of_deriv_var_strings_from_sympyexpr_list(
    list_of_free_symbols, upwind_control_vec
)
print("Derivative variables found in expr:")
print(deriv_vars)

# Now split each derivative into its base gridfunction and derivative operator
(
    list_of_base_gfnames,
    list_of_deriv_ops,
) = fd.extract_base_gfs_and_deriv_ops_lists__from_list_of_deriv_vars(deriv_vars)

print("\nBase gridfunctions appearing inside derivatives:")
print(list_of_base_gfnames)
print("\nDerivative operators:")
print(list_of_deriv_ops)

Derivative variables found in expr:
[vU_dD01, vU_dD20]

Base gridfunctions appearing inside derivatives:
['vU0', 'vU2']

Derivative operators:
['dD1', 'dD0']


# Step 6: Reading gridfunctions from memory with read_gfs_from_memory
### \[Back to [top](#Table-of-Contents)\]

Once the FD coefficients and stencil offsets are known, NRPy must emit C code that:

1. Reads exactly the set of gridfunction values required by all derivatives.
2. Avoids redundant loads.
3. Orders memory accesses to minimize cache misses.
4. Optionally wraps each load in SIMD-aware `ReadSIMD` calls.

The helper

```python
fd.read_gfs_from_memory(
    list_of_base_gridfunction_names_in_derivs,
    fdstencl,
    free_symbols_list,
    mem_alloc_style,
    enable_simd,
)
```

produces C statements of the form

```c
const REAL hDD01_i1m1 = in_gfs[IDX4(HDD01GF, i0, i1-1, i2)];
```

or, if SIMD is enabled and the infrastructure is compatible,

```c
const REAL_SIMD_ARRAY hDD01_i1m1 = ReadSIMD(&in_gfs[IDX4(HDD01GF, i0, i1-1, i2)]);
```

where the suffix on the variable name encodes the offsets in each direction.

In [6]:
# Step 6: Example use of read_gfs_from_memory

# Reset and register some gridfunctions
gri.glb_gridfcs_dict.clear()
par.set_parval_from_str("Infrastructure", "BHaH")

vU = gri.register_gridfunctions_for_single_rank1("vU", group="EVOL")
hDD = gri.register_gridfunctions_for_single_rank2("hDD", group="EVOL", symmetry="sym01")

# Declare derivative symbols
hDD_dD = ixp.declarerank3("hDD_dD", symmetry="sym01")
hDD_dupD = ixp.declarerank3("hDD_dupD", symmetry="sym01")

# Register some code parameters to mix into expressions
a0, a1, b, c = par.register_CodeParameters(
    cparam_type="REAL",
    module="finite_difference_demo",
    names=["a0", "a1", "b", "c"],
    defaultvalues=1,
)

# Build expressions that involve derivatives and undifferentiated gridfunctions
exprlist = [
    b * hDD[1][0] + c * hDD_dD[0][1][1],
    c * hDD_dupD[0][2][2] + b * hDD_dupD[0][2][0] + a1 * a0 * vU[1],
]

# Collect free symbols
free_symbols_list = []
for expr in exprlist:
    free_symbols_list.extend(expr.free_symbols)

# Identify derivative variables; here we pass a non-string so that ddnD versions are added
deriv_vars = fd.extract_list_of_deriv_var_strings_from_sympyexpr_list(
    free_symbols_list, upwind_control_vec=sp.Symbol("betaU")
)

(
    list_of_base_gfnames,
    list_of_deriv_ops,
) = fd.extract_base_gfs_and_deriv_ops_lists__from_list_of_deriv_vars(deriv_vars)

print("Derivative operators:")
print(list_of_deriv_ops)
print("\nBase gridfunctions inside those derivatives:")
print(list_of_base_gfnames)

# For each derivative operator, compute FD coefficients and stencils
fd_order = 2  # 2nd order for a shorter stencil in this demo
fdcoeffs_per_op = [[] for _ in list_of_deriv_ops]
fdstencl_per_op = [[[] for _ in range(4)] for __ in list_of_deriv_ops]

for i, deriv_op in enumerate(list_of_deriv_ops):
    fdcoeffs_per_op[i], fdstencl_per_op[i] = fd.compute_fdcoeffs_fdstencl(deriv_op, fd_order)

print("\nFinite difference stencils for each operator:")
print(fdstencl_per_op)

# Generate scalar (non-SIMD) memory access code, with mem_alloc_style = "210"
read_code_scalar = fd.read_gfs_from_memory(
    list_of_base_gfnames,
    fdstencl_per_op,
    free_symbols_list,
    mem_alloc_style="210",
    enable_simd=False,
)
print("\nScalar memory read code:")
print(read_code_scalar)

# Enable SIMD for one of the gridfunctions to show the SIMD path
gri.glb_gridfcs_dict["vU1"].gf_array_name = "simd_in_gfs_vU1"

read_code_simd = fd.read_gfs_from_memory(
    list_of_base_gfnames,
    fdstencl_per_op,
    free_symbols_list,
    mem_alloc_style="012",
    enable_simd=True,
)
print("SIMD memory read code:")
print(read_code_simd)

Derivative operators:
['dD1', 'ddnD0', 'ddnD2', 'dupD0', 'dupD2']

Base gridfunctions inside those derivatives:
['hDD01', 'hDD02', 'hDD02', 'hDD02', 'hDD02']

Finite difference stencils for each operator:
[[[0, -1, 0], [0, 1, 0]], [[-2, 0, 0], [-1, 0, 0], [0, 0, 0]], [[0, 0, -2], [0, 0, -1], [0, 0, 0]], [[0, 0, 0], [1, 0, 0], [2, 0, 0]], [[0, 0, 0], [0, 0, 1], [0, 0, 2]]]

Scalar memory read code:
const REAL hDD01_i1m1 = in_gfs[IDX4(HDD01GF, i0, i1-1, i2)];
const REAL hDD01 = in_gfs[IDX4(HDD01GF, i0, i1, i2)];
const REAL hDD01_i1p1 = in_gfs[IDX4(HDD01GF, i0, i1+1, i2)];
const REAL hDD02_i2m2 = in_gfs[IDX4(HDD02GF, i0, i1, i2-2)];
const REAL hDD02_i2m1 = in_gfs[IDX4(HDD02GF, i0, i1, i2-1)];
const REAL hDD02_i0m2 = in_gfs[IDX4(HDD02GF, i0-2, i1, i2)];
const REAL hDD02_i0m1 = in_gfs[IDX4(HDD02GF, i0-1, i1, i2)];
const REAL hDD02 = in_gfs[IDX4(HDD02GF, i0, i1, i2)];
const REAL hDD02_i0p1 = in_gfs[IDX4(HDD02GF, i0+1, i1, i2)];
const REAL hDD02_i0p2 = in_gfs[IDX4(HDD02GF, i0+2, i1, i2)];
con

# Step 7: Building prototype FD operators with proto_FD_operators_to_sympy_expressions
### \[Back to [top](#Table-of-Contents)\]

The function

```python
fd.proto_FD_operators_to_sympy_expressions(
    list_of_proto_deriv_symbs,
    fd_order,
    fdcoeffs,
    fdstencl,
    enable_simd=False,
)
```

converts a collection of prototype derivative symbols (for example `FDPROTO_dD0`, `FDPROTO_dKOD1`, `FDPROTO_dupD2`) into:

* `FDexprs`: SymPy expressions representing the finite difference formula in terms of temporary symbols like `FDPROTO_i0m1_i1p1`.
* `FDlhsvarnames`: strings that will become the left hand sides of C assignments (including the appropriate type).
* `symbol_to_Rational_dict`: a dictionary mapping numeric constants (like $1/4$ or $3/2$) to named symbols (like `FDPart1_Rational_1_4`), which helps with CSE and SIMD friendly algebra.

The routine also:

* Multiplies by the appropriate inverse grid spacings (`invdxx0`, `invdxx1`, `invdxx2`) based on the derivative directions.
* Handles mixed derivatives (`dDDij`) by multiplying the appropriate inverse spacings.
* Populates a global dictionary `FDFunctions_dict` with `FDFunction` objects, which will later be used to generate standalone C functions.

In [7]:
# Step 7: Example for proto_FD_operators_to_sympy_expressions

# Reset gridfunctions and FDFunctions_dict
gri.glb_gridfcs_dict.clear()
fd.FDFunctions_dict.clear()
par.set_parval_from_str("Infrastructure", "BHaH")

# Register a prototype gridfunction FDPROTO
FDPROTO_gf = gri.register_gridfunctions("FDPROTO")[0]

# Declare various derivative prototypes built on FDPROTO
FDPROTO_dD = ixp.declarerank1("FDPROTO_dD")
FDPROTO_dupD = ixp.declarerank1("FDPROTO_dupD")
FDPROTO_dKOD = ixp.declarerank1("FDPROTO_dKOD")
FDPROTO_dDD = ixp.declarerank2("FDPROTO_dDD", symmetry="sym01")

# Declare an auxiliary vector to test mixing with non-FDPROTO symbols
vU = ixp.declarerank1("vU")

# Some code parameters
a0, a1, b, c = par.register_CodeParameters(
    cparam_type="REAL",
    module="finite_difference_demo",
    names=["a0_FD", "a1_FD", "b_FD", "c_FD"],
    defaultvalues=1,
)

# Build expressions involving multiple FD operators
exprlist = [
    b * FDPROTO_dDD[1][0] + c * FDPROTO_gf - a0 * FDPROTO_dupD[2],
    c * FDPROTO_dKOD[1] + a1 * a0 * vU[1],
]

# Collect free symbols
free_symbols_list = []
for expr in exprlist:
    free_symbols_list.extend(expr.free_symbols)

# Identify derivative prototype symbols; pass a non-string to activate dup/ddn behavior
list_of_proto_deriv_symbs = fd.extract_list_of_deriv_var_strings_from_sympyexpr_list(
    free_symbols_list, upwind_control_vec=sp.Symbol("betaU_proto")
)

print("Prototype derivative symbols:")
print(list_of_proto_deriv_symbs)

# Extract operator substrings like "dDD01", "dKOD1", "ddnD2", "dupD2"
list_of_proto_deriv_ops = [str(item).split("_")[1] for item in list_of_proto_deriv_symbs]
print("\nPrototype derivative operators:")
print(list_of_proto_deriv_ops)

# For each operator, compute FD coefficients and stencils
fd_order = 2
fdcoeffs_per_op = [[] for _ in list_of_proto_deriv_ops]
fdstencl_per_op = [[[] for _ in range(4)] for __ in list_of_proto_deriv_ops]

for i, deriv_op in enumerate(list_of_proto_deriv_ops):
    fdcoeffs_per_op[i], fdstencl_per_op[i] = fd.compute_fdcoeffs_fdstencl(deriv_op, fd_order)

print("\nFinite difference stencils for prototype operators:")
print(fdstencl_per_op)

# Convert FD operators into SymPy expressions (scalar version)
FDexprs, FDlhsvarnames, symbol_to_Rational_dict = fd.proto_FD_operators_to_sympy_expressions(
    list_of_proto_deriv_symbs,
    fd_order,
    fdcoeffs_per_op,
    fdstencl_per_op,
    enable_simd=False,
)

print("\nLeft-hand sides for FD operators (scalar version):")
for lhs in FDlhsvarnames:
    print(lhs)

print("\nFinite difference expressions (scalar version):")
for lhs, expr in zip(FDlhsvarnames, FDexprs):
    print(f"{lhs} = {expr}")

print("\nSymbol to Rational dictionary (numeric constants):")
for symb, rat in symbol_to_Rational_dict.items():
    print(f"{symb} -> {rat}")

# Now build SIMD-aware versions, which will overwrite FDFunctions_dict
fd.FDFunctions_dict.clear()
FDexprs_SIMD, FDlhsvarnames_SIMD, symbol_to_Rational_dict_SIMD = fd.proto_FD_operators_to_sympy_expressions(
    list_of_proto_deriv_symbs,
    fd_order,
    fdcoeffs_per_op,
    fdstencl_per_op,
    enable_simd=True,
)

print("\nSIMD-aware left-hand sides:")
for lhs in FDlhsvarnames_SIMD:
    print(lhs)

Prototype derivative symbols:
[FDPROTO_dDD01, FDPROTO_dKOD1, FDPROTO_ddnD2, FDPROTO_dupD2]

Prototype derivative operators:
['dDD01', 'dKOD1', 'ddnD2', 'dupD2']

Finite difference stencils for prototype operators:
[[[-1, -1, 0], [1, -1, 0], [-1, 1, 0], [1, 1, 0]], [[0, -2, 0], [0, -1, 0], [0, 0, 0], [0, 1, 0], [0, 2, 0]], [[0, 0, -2], [0, 0, -1], [0, 0, 0]], [[0, 0, 0], [0, 0, 1], [0, 0, 2]]]

Left-hand sides for FD operators (scalar version):
const REAL FDPROTO_dDD01
const REAL FDPROTO_dKOD1
const REAL UpwindAlgInputFDPROTO_ddnD2
const REAL UpwindAlgInputFDPROTO_dupD2

Finite difference expressions (scalar version):
const REAL FDPROTO_dDD01 = FDPart1_Rational_1_4*invdxx0*invdxx1*(FDPROTO_i0m1_i1m1 - FDPROTO_i0m1_i1p1 - FDPROTO_i0p1_i1m1 + FDPROTO_i0p1_i1p1)
const REAL FDPROTO_dKOD1 = invdxx1*(-FDPROTO*FDPart1_Rational_3_8 + FDPart1_Rational_1_16*(-FDPROTO_i1m2 - FDPROTO_i1p2) + FDPart1_Rational_1_4*(FDPROTO_i1m1 + FDPROTO_i1p1))
const REAL UpwindAlgInputFDPROTO_ddnD2 = invdxx2*(FDPROT

# Step 8: Wrapping FD expressions into reusable C functions
### \[Back to [top](#Table-of-Contents)\]

Each entry of `FDFunctions_dict` created by `proto_FD_operators_to_sympy_expressions` is an instance of the `FDFunction` class:

```python
class FDFunction:
    def __init__(self, fp_type_alias, fd_order, operator,
                 symbol_to_Rational_dict, FDexpr, enable_simd):
        ...
```

This class stores

* The floating point type alias (`REAL`, `CCTK_REAL`, or `REAL_SIMD_ARRAY`).
* The FD order.
* The operator name (such as `"dDD01"` or `"dupD2"`).
* The symbolic finite difference expression `FDexpr`, already preprocessed and factorized by `proto_FD_operators_to_sympy_expressions`.
* Whether SIMD should be used.

It provides:

* `c_function_name`: an automatically constructed name, such as `fd_function_dDD01_fdorder2` or `SIMD_fd_function_dupD2_fdorder4`.
* `c_function_call(gf_name, deriv_var)`: a helper that emits the C expression for calling the FD function for a given gridfunction name and derivative variable.
* `CFunction_fd_function(FDexpr_c_code)`: which wraps a C code snippet `FDexpr_c_code` (assigning to `FD_result`) into a complete C function using `nrpy.c_function.CFunction`.

The important detail is that `finite_difference.py` does not automatically generate C code strings. After Step 7, each `FDFunction` object knows the SymPy expression `FDexpr`, but its `.CFunction` attribute is not yet initialized. We must:

1. Loop over `fd.FDFunctions_dict`.
2. For each `fd_func`, convert `fd_func.FDexpr` into a C assignment string for `FD_result` using `sp.ccode` directly (no additional CSE).
3. Call `fd_func.CFunction_fd_function(assignment_string)` and assign the result to `fd_func.CFunction`.
4. Only then call `fd.construct_FD_functions_prefunc()` to obtain the full C function definitions as a single prefunc string.

This avoids redundant preprocessing and ensures that the `CFunction` objects are fully initialized before we ask for their combined C code.

In [8]:
# Step 8: Generate full C functions from FDFunctions_dict

prefunc_c_code = ""

# Initialize the CFunction for each stored FDFunction
for op_name, fd_func in fd.FDFunctions_dict.items():
    # Convert the stored FDexpr directly into C code.
    # proto_FD_operators_to_sympy_expressions has already handled factorization and CSE.
    FDexpr_c_str = f"{fd_func.fp_type_alias} FD_result = {sp.ccode(fd_func.FDexpr)};"

    # Wrap into a CFunction and store it on the FDFunction object
    fd_func.CFunction = fd_func.CFunction_fd_function(FDexpr_c_str)

# Now use the helper to assemble full function definitions
prefunc_c_code = fd.construct_FD_functions_prefunc(cfunc_decorators="")

print("Generated C prefunc code for all FD operators:\n")
print(prefunc_c_code)

Generated C prefunc code for all FD operators:

/**
 * Finite difference function for operator dDD01, with FD accuracy order 2.
 */
static NO_INLINE REAL_SIMD_ARRAY SIMD_fd_function_dDD01_fdorder2(const REAL_SIMD_ARRAY FDPROTO_i0m1_i1m1, const REAL_SIMD_ARRAY FDPROTO_i0m1_i1p1,
                                                                 const REAL_SIMD_ARRAY FDPROTO_i0p1_i1m1, const REAL_SIMD_ARRAY FDPROTO_i0p1_i1p1,
                                                                 const REAL_SIMD_ARRAY invdxx0, const REAL_SIMD_ARRAY invdxx1) {
  REAL_SIMD_ARRAY FD_result = FDPart1_Rational_1_4 * invdxx0 * invdxx1 *
                              (FDPROTO_i0m1_i1m1 + FDPROTO_i0p1_i1p1 + FDPart1_NegativeOne_ * (FDPROTO_i0m1_i1p1 + FDPROTO_i0p1_i1m1));
  return FD_result;
} // END FUNCTION SIMD_fd_function_dDD01_fdorder2
/**
 * Finite difference function for operator dKOD1, with FD accuracy order 2.
 */
static NO_INLINE REAL_SIMD_ARRAY SIMD_fd_function_dKOD1_fdorder2(const REAL_SIMD_A

In [9]:
# Step 8 (continued): Example of calling one of the generated FD functions in C

# Choose one operator (for example, the first one in the dictionary)
if fd.FDFunctions_dict:
    example_op = next(iter(fd.FDFunctions_dict.keys()))
    example_fd_func = fd.FDFunctions_dict[example_op]
    print(f"Example operator: {example_op}")
    print(f"C function name: {example_fd_func.c_function_name}")

    # Build a sample function call for gridfunction name "FDPROTO"
    example_call = example_fd_func.c_function_call(gf_name="FDPROTO", deriv_var=f"{example_op}_at_i")
    print("\nSample C function call:")
    print(f"{example_call};")

Example operator: dDD01
C function name: SIMD_fd_function_dDD01_fdorder2

Sample C function call:
const REAL_SIMD_ARRAY dDD01_at_i = SIMD_fd_function_dDD01_fdorder2(FDPROTO_i0m1_i1m1,FDPROTO_i0m1_i1p1,FDPROTO_i0p1_i1m1,FDPROTO_i0p1_i1p1,invdxx0,invdxx1);
